In [2]:
import pandas as pd
# import nbinteract as nbi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, fixed

%matplotlib inline

In [175]:
all_data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
all_data = all_data.replace('North Macedonia', 'Macedonia')
all_data = all_data.drop(columns=['Province/State', 'Lat', 'Long'], axis=1)
all_data = all_data.groupby(['Country/Region']).sum()
all_data.reset_index(level=0, inplace=True)

In [177]:
def get_data_by_country_name(data, country_column, country_name):
    return data.loc[data[country_column] == country_name]


def convert_data_in_day_count_larger_than_zero_format(data, id_vars, value_vars, var_name, value_name):
    converted = pd.melt(data, id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)
    converted = converted.loc[converted[value_name] > 0].reset_index(drop=True)
    converted.insert(0, 'artificial_day', range(1, 1 + len(converted)))
    return converted.loc[converted[value_name] > 0].reset_index(drop=True)

def get_data_for_country(data, name):
    res = get_data_by_country_name(data=data, country_column='Country/Region', country_name=name)
    res = convert_data_in_day_count_larger_than_zero_format(data=res, id_vars=res.columns[ : 1], value_vars=res.columns[2:], var_name='date', value_name='count')
    return res


In [195]:
def get_data_for_countries(data, countries):
    data = [get_data_for_country(data, country) for country in countries]
#     interesting_data = pd.concat([country_data.iloc[:comparable_days_count] for country_data in data])
    return data

def draw_line_plot(day_range, countries, data):
    countries = list(countries)
    if 'Macedonia' not in countries:
        countries.insert(0, 'Macedonia')
    figure = plt.figure(num='figure', figsize=(15,10))
    from_day = day_range[0]
    to_day = day_range[1]
    data = get_data_for_countries(data, countries)
    interesting_data = pd.concat([country_data.loc[(country_data['artificial_day'] >= from_day) & 
                                                   (country_data['artificial_day'] <= to_day)] 
                                  for country_data in data])

    ax = plt.axes()
    line_plot = sns.lineplot(x="artificial_day", y="count", hue='Country/Region',
                             data=interesting_data, ax = ax)

    line_plot.set(xlabel='Релативни денови од кога се појавил првиот случај во таа држава',
                  ylabel='Број на заразени')
    ax.legend().texts[0].set_text("Држава:")
    line_plot.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    line_plot.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    
    plt.show()
    
    return line_plot


In [197]:
style = {'description_width': 'initial'}
layout = widgets.Layout(display='flex',
                justify_content='center',
                width='70%')


day_range_chooser = widgets.IntRangeSlider(
    value=[1, 20],
    min=1,
    max=len(all_data.columns) - 4,
    step=1,
    description='Изберете ранг на денови:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style,
    layout=layout
)

country_selector = widgets.SelectMultiple(
    options=all_data['Country/Region'].unique(),
    value=['Macedonia', 'Serbia', 'Albania', 'Greece', 'Bulgaria'],
    description='Изберете држави што сакате да ги споредите:',
    disabled=False,
    style = style,
    layout=layout
)


p = interact(draw_line_plot, day_range=day_range_chooser, countries=country_selector, data=fixed(all_data))

interactive(children=(IntRangeSlider(value=(1, 20), continuous_update=False, description='Изберете ранг на ден…